In [4]:
## iterators
mytuple = ("apple", "banana", "cherry")
myit = iter(mytuple)
next(myit)


# __iter__ method works like __init__, but the variable is iterable
class MyNumbers:
  def __iter__(self):
    self.a = 1
    return self

  def __next__(self):
    x = self.a
    self.a += 1
    return x

myclass = MyNumbers()
myiter = iter(myclass)

print(next(myiter))
print(next(myiter))
print(next(myiter))
print(next(myiter))
print(next(myiter))


## raise StopIteration to make it loop up to a point with 'for'
class MyNumbers:
  def __iter__(self):
    self.a = 1
    return self

  def __next__(self):
    if self.a <= 20:
      x = self.a
      self.a += 1
      return x
    else:
      raise StopIteration

myclass = MyNumbers()
myiter = iter(myclass)

for x in myiter:
  print(x)

1
2
3
4
5
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20


In [10]:
# creating a generator by using 'yield' as part of __iter__
class MyIterable:
    def __init__(self, data):
        self.data = data

    def __iter__(self):
        for x in self.data:
            yield x
            

my_iterable = MyIterable([1, 2, 3])
for x in my_iterable:
    print(x)

1
2
3


In [31]:
'aioho422'.zfill(100)

'00000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000aioho422'

In [33]:
v = memoryview(b'abcefg')
for l in v:
    print(l)

97
98
99
101
102
103


In [35]:
## memoryview
import array
a = array.array('l', [-11111111, 22222222, -33333333, 44444444])
m = memoryview(a)
m[1]



22222222

In [36]:
# memoryview object can be hashed
v = memoryview(b'abcefg')
hash(v) == hash(b'abcefg')

True

In [38]:
b"abc".hex()

'616263'

In [47]:
my_bytearray = bytearray(b'Hello, world!')
mv = memoryview(my_bytearray)

# Do something with the memoryview
mv[0]=101

# Release the memory associated with the memoryview
mv.release()

# view change
my_bytearray

72


bytearray(b'eello, world!')

In [56]:
hash(frozenset([1,2,3]))

-272375401224217160

In [65]:
d = {"one": 1, "two": 2, "three": 3, "four": 4}
for i in reversed(d.items()):
    print(i)
    d[i[0]] = 3
d

('four', 4)
('three', 3)
('two', 2)
('one', 1)


{'one': 3, 'two': 3, 'three': 3, 'four': 3}

In [66]:
for k,v in d.items():
    d[k] = 2
d

{'one': 2, 'two': 2, 'three': 2, 'four': 2}

In [74]:
type(d.keys())

dict_keys

In [76]:
# confirm d (a dict) is a mapping object
import collections.abc
isinstance(d, collections.abc.Mapping)

True

In [97]:
# Create a custom mapping object: ie a dictionary with extra methods
class MyMapping(collections.abc.Mapping):
    def __init__(self, data):
        self._data = data

    def __getitem__(self, key):
        return self._data[key]
    
    def __setitem__(self, key, newdata):
        self._data[key] = newdata
    
    def __iter__(self):
        for x in self._data:
            yield x
            
    def __len__(self):
        l = 0
        for i in self:
            l += 1
        return l
    
k = MyMapping({1:2, 3:4})
print(len(k))
print(k[3])
for v in k.items():
    print(v)
k[2] = 5
print(k[2])

2
4
(1, 2)
(3, 4)
5


In [98]:
type(Ellipsis)()

Ellipsis

In [99]:
Ellipsis

Ellipsis

In [ ]:
### Can use Ellipsis in slicing and indexing: doesn't seem useful
my_list = [1, 2, 3, 4, 5, 6]

# Use Ellipsis to include all elements up to the fourth element
print(my_list[:Ellipsis, 4])  # [1, 2, 3, 4]

# Use Ellipsis to include all elements after the second element
print(my_list[2, Ellipsis])  # [3, 4, 5, 6]

# Use Ellipsis to include all elements
print(my_list[Ellipsis])  # [1, 2, 3, 4, 5, 6]


In [103]:
## NotImplemented shows user that method isn't made: inheriting class might want to create it

class MyNumber:
    def __add__(self, other):
        # Implement __add__
        pass

    def __radd__(self, other):
        # Return NotImplemented to delegate __radd__ to another object
        return NotImplemented

# Define a class that implements __radd__
class MyOtherNumber:
    def __radd__(self, other):
        # Implement __radd__
        pass

MyNumber() + 2

In [106]:
class MyNumber:
    def __init__(self, value):
        self.value = value

    def __radd__(self, other):
        # Return the sum of the other object and the value of the MyNumber object
        return other + self.value

# Create an object of the MyNumber class
my_number = MyNumber(10)

# this works
5 + my_number

# this would need __add__: my_number + 5


15

In [3]:
# an async contextlib
import contextlib
import asyncio
import requests
@contextlib.asynccontextmanager
async def get_google():
    # Connect to the database
    vals = requests.get('http://www.google.com')
    try:
        # Yield the connection
        yield vals
    finally:
        # Close the connection
        print('closed')
async def main():
    # Use the context manager to get a database connection
    async with get_google() as google_txt:
        print(google_txt.text)   
# run one of the below 2 lines: need to start a new event loop if you've run loop.close()
loop = asyncio.get_event_loop()
loop = asyncio.new_event_loop()
loop.run_until_complete(main())
loop.close()

<coroutine object main at 0x7fe2800215c0>

In [8]:
# this is a perfectly fine way to make a custom exception
class MyException(Exception):
    """Docstring for what the exception means"""

raise MyException({"message":"My hovercraft is full of animals", "animal":"eels"})


MyException: {'message': 'My hovercraft is full of animals', 'animal': 'eels'}

In [3]:
# more complicated way https://stackoverflow.com/questions/1319615/proper-way-to-declare-custom-exceptions-in-modern-python
try:
    raise MyException({"message":"My hovercraft is full of animals", "animal":"eels"})
except MyException as e:
    details = e.args[0]
    print(details["animal"])


eels


In [17]:
### Q: why make custom exception class? 
### A: more descriptive and meaningful error message; make the error specific to your library

# how to make a custom exception class?
class MyAppValueError(ValueError):
    '''Raise when a specific subset of values in context of app is wrong'''
    def __init__(self, message, foo, *args):
        self.message = message # without this you may get DeprecationWarning
        # Special attribute you desire with your Error, 
        # perhaps the value that caused the error?:
        self.foo = foo         
        # allow users initialize misc. arguments as any other builtin Error
        super(MyAppValueError, self).__init__(message, foo, *args) 

raise MyAppValueError('msg for user', 'something else', 'wasteman')


MyAppValueError: ('msg for user', 'something else', 'wasteman')

In [24]:
class NetworkError(Exception):
    def __init__(self, message, errors):
        super().__init__(message)
        self.errors = errors
raise NetworkError('A network error occurred', ['Error 1', 'Error 2'])

NetworkError: A network error occurred

In [38]:
# can catch multiple types of error and get info on the error
try:
    int('asfg'+'aa'+'3f2w4g')
except (RuntimeError, TypeError, NameError, ValueError) as e:
    print(type(e))
    print(e.args)
    print('value was asfg')
    print(e) 

<class 'ValueError'>
("invalid literal for int() with base 10: 'asfgaa3f2w4g'",)
value was asfg
invalid literal for int() with base 10: 'asfgaa3f2w4g'


In [39]:
# try/except can catch errors which propagate from called functions
def this_fails():
    x = 1/0
try:
    this_fails()
except ZeroDivisionError as err:
    print('Handling run-time error:', err)

Handling run-time error: division by zero


finally is executed regardless of whether the statements in the try block fail or succeed. else is executed only if the statements in the try block don't raise an exception.

https://stackoverflow.com/questions/6051934/purpose-of-else-and-finally-in-exception-handling

In [84]:
# if open() in try fails, then exception is run, then runtime returns to rest of try statement: this is 
# true for OSError but not KeyError, which if called *does* skip the else section
for arg in sys.argv[1:]:
    f = None
    d = {4:3}
    try:
        f = open(arg, 'r')
        print('still running after OSError is raised!' + d[3])
        print('a')
    except OSError:
        print('cannot open', arg)
    except KeyError as e:
        print(e)
    else:
        print('hahaha')   
        print(arg, 'has', len(f.readlines()), 'lines')
    finally:
        #print(arg, 'has', len(f.readlines()), 'lines')
        print('ho')
        if f:
            f.close()


cannot open -f
ho
a
hahaha
/Users/adambricknell/Library/Jupyter/runtime/kernel-baf82497-861a-4941-8c79-854ddfa5ff82.json has 12 lines
ho


In [103]:
# getting info on specific error raised. Also can store 'context' for later
import traceback
d={4:3}
try:
    d[3]
except Exception as e:
    context = {
        'error_type': e.__class__.__name__,
        'error_message': str(e),
        'error_traceback': traceback.format_exc(),
        'error_keys':dir(e),
        'args':e.args
    }
    raise Exception(context)


Exception: {'error_type': 'KeyError', 'error_message': '3', 'error_traceback': 'Traceback (most recent call last):\n  File "/var/folders/x2/bt81rqpj7pl_j7fczgml3pd80000gn/T/ipykernel_11440/871524406.py", line 5, in <module>\n    d[3]\nKeyError: 3\n', 'error_keys': ['__cause__', '__class__', '__context__', '__delattr__', '__dict__', '__dir__', '__doc__', '__eq__', '__format__', '__ge__', '__getattribute__', '__gt__', '__hash__', '__init__', '__init_subclass__', '__le__', '__lt__', '__ne__', '__new__', '__reduce__', '__reduce_ex__', '__repr__', '__setattr__', '__setstate__', '__sizeof__', '__str__', '__subclasshook__', '__suppress_context__', '__traceback__', 'args', 'with_traceback'], 'args': (3,)}